In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed
import os
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Any

seed = 18022004
np.random.seed(seed)
set_seed(seed)

/drive2/phatnt/zTrans/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_prefix: str = 'data'
repo_prefix: str = f'{data_prefix}/repos'

model_id: str = 'deepseek-ai/deepseek-coder-6.7b-instruct'

In [15]:
data_name = 'data_method_total_treesitter.parquet'

data_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{data_name}', engine = 'pyarrow')

In [16]:
data_df

,id,fromLib,toLib,repoName,prevCommit,startCommit,endCommit,fileName,startCommitChanges,endCommitChanges,startCode_cleaned,endCode_cleaned,total_methods_before,total_methods_after,method_before,method_after,method_diff,method_name,method_signature
0,0,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/main/java/org/drools/rule/...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.rule.builder.dialect.java;\...,package org.drools.rule.builder.dialect.java;\...,5,5,private void rewriteModify(final RuleBuildCont...,private void rewriteModify(final RuleBuildCont...,"--- \n+++ \n@@ -50,8 +50,6 @@\n consequence.ap...",rewriteModify,private rewriteModify(final RuleBuildContext ...
1,1,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/guvno...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.guvnor.server.util;\nimport...,package org.drools.guvnor.server.util;\nimport...,4,4,public void testAddSentenceMultipleTypes() {\n...,public void testAddSentenceMultipleTypes() {\n...,"--- \n+++ \n@@ -1,27 +1,25 @@\n public void te...",testAddSentenceMultipleTypes,public testAddSentenceMultipleTypes()
2,2,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.integrationtests;\nimport j...,package org.drools.integrationtests;\nimport j...,35,35,public void testLockOnActiveWithModify2() thro...,public void testLockOnActiveWithModify2() thro...,"--- \n+++ \n@@ -70,9 +70,8 @@\n session.getAge...",testLockOnActiveWithModify2,public testLockOnActiveWithModify2()
3,3,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.integrationtests;\nimport j...,package org.drools.integrationtests;\nimport j...,35,35,public void testUpdateActivationCreationNoLoop...,public void testUpdateActivationCreationNoLoop...,"--- \n+++ \n@@ -52,8 +52,8 @@\n item );\n \n \...",testUpdateActivationCreationNoLoop,public testUpdateActivationCreationNoLoop()
4,4,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.integrationtests;\nimport j...,package org.drools.integrationtests;\nimport s...,198,198,public void testJoinNodeModifyObject() throws ...,public void testJoinNodeModifyObject() throws ...,"--- \n+++ \n@@ -9,8 +9,6 @@\n ruleBase.addPack...",testJoinNodeModifyObject,public testJoinNodeModifyObject()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81799,81799,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yandex-qatools_hamcrest-pojo-matcher-generator,13ac1e25cc715b6855890070099a3cdbea36cf56,cc7a474d3ea53164d28b4a1b23c25989982e7d03,cc7a474d3ea53164d28b4a1b23c25989982e7d03,feature-matcher-generator/src/test/java/ru/yan...,+ru.qatools.commons:properties\n-ru.yandex.qat...,+ru.qatools.commons:properties\n-ru.yandex.qat...,package ru.yandex.qatools.processors.matcher.g...,package ru.yandex.qatools.processors.matcher.g...,2,2,"@Parameterized.Parameters(name = ""{0}"")\n p...","@Parameterized.Parameters(name 

In [18]:
# remove all special cases
# special cases are neither methods_before nor methods_after is blank
filtered_df: pd.DataFrame = data_df[data_df['method_before'].str.len() > 0]
# print(len(filtered_df))
filtered_df: pd.DataFrame = filtered_df[filtered_df['method_after'].str.len() > 0]
print(len(filtered_df))

data_df = filtered_df

38393


In [19]:
data_df

,id,fromLib,toLib,repoName,prevCommit,startCommit,endCommit,fileName,startCommitChanges,endCommitChanges,startCode_cleaned,endCode_cleaned,total_methods_before,total_methods_after,method_before,method_after,method_diff,method_name,method_signature
0,0,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/main/java/org/drools/rule/...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.rule.builder.dialect.java;\...,package org.drools.rule.builder.dialect.java;\...,5,5,private void rewriteModify(final RuleBuildCont...,private void rewriteModify(final RuleBuildCont...,"--- \n+++ \n@@ -50,8 +50,6 @@\n consequence.ap...",rewriteModify,private rewriteModify(final RuleBuildContext ...
1,1,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/guvno...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.guvnor.server.util;\nimport...,package org.drools.guvnor.server.util;\nimport...,4,4,public void testAddSentenceMultipleTypes() {\n...,public void testAddSentenceMultipleTypes() {\n...,"--- \n+++ \n@@ -1,27 +1,25 @@\n public void te...",testAddSentenceMultipleTypes,public testAddSentenceMultipleTypes()
2,2,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.integrationtests;\nimport j...,package org.drools.integrationtests;\nimport j...,35,35,public void testLockOnActiveWithModify2() thro...,public void testLockOnActiveWithModify2() thro...,"--- \n+++ \n@@ -70,9 +70,8 @@\n session.getAge...",testLockOnActiveWithModify2,public testLockOnActiveWithModify2()
3,3,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.integrationtests;\nimport j...,package org.drools.integrationtests;\nimport j...,35,35,public void testUpdateActivationCreationNoLoop...,public void testUpdateActivationCreationNoLoop...,"--- \n+++ \n@@ -52,8 +52,8 @@\n item );\n \n \...",testUpdateActivationCreationNoLoop,public testUpdateActivationCreationNoLoop()
4,4,ant:ant,org.apache.ant:ant,bobmcwhirter_drools,2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f,92384035a8651b675c82689b24837eb8adb81d66,0df25aead759b3b542a685ac21a3a009bcd22fe9,drools-compiler/src/test/java/org/drools/integ...,+org.mockito:mockito-all,-org.jmock:jmock\n-org.jmock:jmock-legacy,package org.drools.integrationtests;\nimport j...,package org.drools.integrationtests;\nimport s...,198,198,public void testJoinNodeModifyObject() throws ...,public void testJoinNodeModifyObject() throws ...,"--- \n+++ \n@@ -9,8 +9,6 @@\n ruleBase.addPack...",testJoinNodeModifyObject,public testJoinNodeModifyObject()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81799,81799,ru.yandex.qatools.properties:properties-loader,ru.qatools.commons:properties,yandex-qatools_hamcrest-pojo-matcher-generator,13ac1e25cc715b6855890070099a3cdbea36cf56,cc7a474d3ea53164d28b4a1b23c25989982e7d03,cc7a474d3ea53164d28b4a1b23c25989982e7d03,feature-matcher-generator/src/test/java/ru/yan...,+ru.qatools.commons:properties\n-ru.yandex.qat...,+ru.qatools.commons:properties\n-ru.yandex.qat...,package ru.yandex.qatools.processors.matcher.g...,package ru.yandex.qatools.processors.matcher.g...,2,2,"@Parameterized.Parameters(name = ""{0}"")\n p...","@Parameterized.Parameters(name 

In [27]:
# check if row with the same 'fileName' have same 'total_methods_before' and 'total_methods_after'
# Check if all rows with the same 'fileName' and 'startCommit' have the same 'total_methods_before' and 'total_methods_after'
inconsistent_groups = data_df.groupby(['fileName', 'startCommit', 'endCommit']).apply(
    lambda x: not (
        x['total_methods_before'].nunique() == 1 and
        x['total_methods_after'].nunique() == 1
    )
)

# Filter inconsistent groups
inconsistent_pairs = inconsistent_groups[inconsistent_groups].index.tolist()

if not inconsistent_pairs:
    print("All groups have matching 'total_methods_before' and 'total_methods_after'.")
else:
    print(len(inconsistent_pairs), "inconsistent groups found.")
    print("Inconsistent groups:", inconsistent_pairs)

All groups have matching 'total_methods_before' and 'total_methods_after'.


/tmp/ipykernel_1751119/1172154008.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  inconsistent_groups = data_df.groupby(['fileName', 'startCommit', 'endCommit']).apply(


BY MIGRATION

In [28]:
unique_commit_method_mapping = (
    data_df.groupby(['startCommit', 'endCommit'])
    .agg(repoName=('repoName', 'first'), count=('repoName', 'size'))
    .reset_index()
    .sort_values(by='count', ascending=False)
)

unique_commit_method_mapping

,startCommit,endCommit,repoName,count
43,0e092deee046995b2813e4a130cb61592d9efd84,3cfa7b24ee562abfabdf19bc1035c61c3ed3eff2,ProgrammeVitam_vitam,2650
703,d28d8235630d3a5d2940598a8fd562039a078c57,a881c12aae720a817d1cadc393902d2ba583c762,apache_groovy,2644
718,d68640c5a7995fbdbe0aa61dc88b0f0b23b6d5fe,3cfa7b24ee562abfabdf19bc1035c61c3ed3eff2,ProgrammeVitam_vitam,2505
168,2d79c0eb00e37f5ec35651936fdb66ab5ba095a5,a87749a6f89d97fe166c2152646c680214bf11f3,ModeShape_modeshape,1521
711,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,mulesoft_mule,1335
...,...,...,...,...
801,f3adc7b225dd94e55a7e848baaac972e2e95a663,f3adc7b225dd94e55a7e848baaac972e2e95a663,ldp4j_ldp4j,1
815,f94baf6a10b287a4c89fa560da1e96a8cce163b6,f94baf6a10b287a4c89fa560da1e96a8cce163b6,javaee_glassfish,1
810,f72721587a3a37e3a2f9d478e0860fdaedb7c645,f72721587a3a37e3a2f9d478e0860fdaedb7c645,umlet_umlet,1
806,f5bbbf16d46ded9026d240acc603a78b1350fe15,f5bbbf16d46ded9026d240acc603a78b1350fe15,adobe_aem-core-wcm-components,1


In [ ]:
unique_commit_method_mapping.describe()

,count
count,839.000000
mean,45.760429
std,192.675190
min,1.000000
25%,2.000000
50%,7.000000
75%,22.500000
max,2650.000000


In [44]:
# Group by 'startCommit' and 'endCommit' and calculate the required aggregations
unique_commit_method_mapping = (
    data_df.groupby(['startCommit', 'endCommit'])
    .agg(
        repoName=('repoName', 'first'),
        count=('repoName', 'size'),  # The number of rows (method change count)
        total_methods_before_sum=('total_methods_before', 'sum'),
        total_methods_after_sum=('total_methods_after', 'sum')
    )
    .reset_index()
    .sort_values(by='count', ascending=False)
)

# Calculate the percentage of method changes based on the 'count' (number of changes)
unique_commit_method_mapping['method_change_percentage_before'] = (
    (unique_commit_method_mapping['count'] / unique_commit_method_mapping['total_methods_before_sum']) * 100
)

unique_commit_method_mapping['method_change_percentage_after'] = (
    (unique_commit_method_mapping['count'] / unique_commit_method_mapping['total_methods_after_sum']) * 100
)

# Display the result with relevant columns
result = unique_commit_method_mapping[['startCommit', 'endCommit', 'repoName', 'count', 'method_change_percentage_before', 'method_change_percentage_after']]

result

,startCommit,endCommit,repoName,count,method_change_percentage_before,method_change_percentage_after
43,0e092deee046995b2813e4a130cb61592d9efd84,3cfa7b24ee562abfabdf19bc1035c61c3ed3eff2,ProgrammeVitam_vitam,2650,3.584617,2.735682
703,d28d8235630d3a5d2940598a8fd562039a078c57,a881c12aae720a817d1cadc393902d2ba583c762,apache_groovy,2644,1.175372,1.005461
718,d68640c5a7995fbdbe0aa61dc88b0f0b23b6d5fe,3cfa7b24ee562abfabdf19bc1035c61c3ed3eff2,ProgrammeVitam_vitam,2505,4.493354,3.412109
168,2d79c0eb00e37f5ec35651936fdb66ab5ba095a5,a87749a6f89d97fe166c2152646c680214bf11f3,ModeShape_modeshape,1521,1.855919,1.603974
711,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,mulesoft_mule,1335,6.065700,5.558563
...,...,...,...,...,...,...
801,f3adc7b225dd94e55a7e848baaac972e2e95a663,f3adc7b225dd94e55a7e848baaac972e2e95a663,ldp4j_ldp4j,1,8.333333,8.333333
815,f94baf6a10b287a4c89fa560da1e96a8cce163b6,f94baf6a10b287a4c89fa560da1e96a8cce163b6,javaee_glassfish,1,2.500000,2.500000
810,f72721587a3a37e3a2f9d478e0860fdaedb7c645,f72721587a3a37e3a2f9d478e0860fdaedb7c645,umlet_umlet,1,10.000000,10.000000
806,f5bbbf16d46ded9026d240acc603a78b1350fe15,f5bbbf16d46ded9026d240acc603a78b1350fe15,adobe_aem-core-wcm-components,1,3.846154,3.846154


In [37]:
unique_commit_method_mapping.describe()

,count,total_methods_before_sum,total_methods_after_sum,method_change_percentage_before,method_change_percentage_after
count,839.000000,839.000000,839.000000,839.000000,839.000000
mean,45.760429,1261.303933,1382.462455,16.748704,16.856890
std,192.675190,9295.960909,10894.028407,19.287485,19.817816
min,1.000000,1.000000,1.000000,0.256361,0.256148
25%,2.000000,17.000000,17.000000,6.043972,5.886758
50%,7.000000,73.000000,72.000000,10.526316,10.526316
75%,22.500000,285.000000,291.000000,18.933824,18.750000
max,2650.000000,224950.000000,262964.000000,100.000000,100.000000


In [13]:
# Group by 'startCommit' and 'endCommit' and count the number of unique 'fileName' entries in each group
unique_commit_class_mapping = (
    data_df.groupby(['startCommit', 'endCommit'])
    .agg(repoName=('repoName', 'first'), file_count=('fileName', 'nunique'))
    .reset_index()
    .sort_values(by='file_count', ascending=False)
)

unique_commit_class_mapping

,startCommit,endCommit,repoName,file_count
711,d59c7d6f4485bdd578ee5c45dfc421a705d48bc0,2a903b90e3a5fb015d59ccef4ccdde265b1746d8,mulesoft_mule,649
718,d68640c5a7995fbdbe0aa61dc88b0f0b23b6d5fe,3cfa7b24ee562abfabdf19bc1035c61c3ed3eff2,ProgrammeVitam_vitam,498
43,0e092deee046995b2813e4a130cb61592d9efd84,3cfa7b24ee562abfabdf19bc1035c61c3ed3eff2,ProgrammeVitam_vitam,480
830,fce9eff724e31ee16deb995f5427dce3e90e8238,fce9eff724e31ee16deb995f5427dce3e90e8238,MyCollab_mycollab,454
428,748c1bd2fa2942459e88e26e9820ebe3b3dd1fb5,748c1bd2fa2942459e88e26e9820ebe3b3dd1fb5,apache_hive,401
...,...,...,...,...
801,f3adc7b225dd94e55a7e848baaac972e2e95a663,f3adc7b225dd94e55a7e848baaac972e2e95a663,ldp4j_ldp4j,1
5,02c74f96eee9f82849cc236f98db53f9b8c7d4a6,02c74f96eee9f82849cc236f98db53f9b8c7d4a6,cloudfoundry_cf-java-client,1
810,f72721587a3a37e3a2f9d478e0860fdaedb7c645,f72721587a3a37e3a2f9d478e0860fdaedb7c645,umlet_umlet,1
806,f5bbbf16d46ded9026d240acc603a78b1350fe15,f5bbbf16d46ded9026d240acc603a78b1350fe15,adobe_aem-core-wcm-components,1


In [14]:
unique_commit_class_mapping.describe()

,file_count
count,839.000000
mean,14.835518
std,47.900395
min,1.000000
25%,2.000000
50%,4.000000
75%,10.000000
max,649.000000


In [46]:
# Group by 'fileName' and calculate the required aggregations
unique_file_method_mapping = (
    data_df.groupby(['fileName'])
    .agg(
        repoName=('repoName', 'first'),
        count=('repoName', 'size'),  # The number of rows (method change count)
        total_methods_before_sum=('total_methods_before', 'sum'),
        total_methods_after_sum=('total_methods_after', 'sum')
    )
    .reset_index()
    .sort_values(by='count', ascending=False)
)

# Calculate the percentage of method changes based on the 'count' (number of changes)
unique_file_method_mapping['method_change_percentage_before'] = (
    (unique_file_method_mapping['count'] / unique_file_method_mapping['total_methods_before_sum']) * 100
)

unique_file_method_mapping['method_change_percentage_after'] = (
    (unique_file_method_mapping['count'] / unique_file_method_mapping['total_methods_after_sum']) * 100
)

# Display the result with relevant columns
result = unique_file_method_mapping[['fileName', 'repoName', 'count', 'method_change_percentage_before', 'method_change_percentage_after']]

result


,fileName,repoName,count,method_change_percentage_before,method_change_percentage_after
9751,src/main/org/codehaus/groovy/classgen/AsmClass...,apache_groovy,222,0.684932,0.591716
10091,src/test/org/codehaus/groovy/antlr/treewalker/...,apache_groovy,220,0.558659,0.564972
8544,sources/ihm-demo/ihm-demo-web-application/src/...,ProgrammeVitam_vitam,195,1.296887,1.301301
9775,src/main/org/codehaus/groovy/runtime/DefaultGr...,apache_groovy,184,0.215983,0.166667
5053,modeshape-jcr/src/test/java/org/modeshape/jcr/...,ModeShape_modeshape,177,0.460829,0.355872
...,...,...,...,...,...
4853,mockserver-core/src/main/java/org/mockserver/p...,mock-server_mockserver,1,100.000000,100.000000
2092,correlation/src/main/java/com/networknt/correl...,networknt_light-4j,1,20.000000,20.000000
4842,mockserver-core/src/main/java/org/mockserver/c...,mock-server_mockserver,1,16.666667,16.666667
2105,dalesbred/src/test/java/fi/evident/dalesbred/D...,EvidentSolutions_dalesbred,1,100.000000,100.000000
